In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import matplotlib.pyplot as plt

In [2]:
# GPU Configuration
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [4]:
training_set = train_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7518 images belonging to 6 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_set = test_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7518 images belonging to 6 classes.


In [6]:
# Custom F1 Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall for each batch
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Calculate F1 score as the harmonic mean of precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [7]:
# Build the CNN-LSTM Model
cnn_lstm = tf.keras.models.Sequential()

In [8]:
# CNN Layers for Feature Extraction
cnn_lstm.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', strides=2, padding="same", input_shape=[64, 64, 3]))
cnn_lstm.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn_lstm.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
cnn_lstm.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Flatten and Reshape for LSTM
cnn_lstm.add(tf.keras.layers.Flatten())
cnn_lstm.add(tf.keras.layers.Reshape((16, -1)))  # Reshape to (time_steps, features)

In [10]:
# LSTM Layer
cnn_lstm.add(tf.keras.layers.LSTM(units=64, activation='tanh', return_sequences=False))

In [11]:
# Fully Connected Layers
cnn_lstm.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn_lstm.add(tf.keras.layers.Dense(6, activation='softmax'))  # 6 classes

In [12]:
# Compile the model with Precision, Recall, and F1 Score
cnn_lstm.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1_score')  # Add custom F1 score metric
    ]
)

In [13]:
# Train the model
history = cnn_lstm.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=100
)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.3591 - f1_score: 0.0280 - loss: 1.6298 - precision: 0.2834 - recall: 0.0149

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


235/235 ━━━━━━━━━━━━━━━━━━━━ 71s 288ms/step - accuracy: 0.3592 - f1_score: 0.0282 - loss: 1.6297 - precision: 0.2844 - recall: 0.0150 - val_accuracy: 0.4166 - val_f1_score: 0.1999 - val_loss: 1.5092 - val_precision: 0.5730 - val_recall: 0.1210
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 54s 229ms/step - accuracy: 0.4312 - f1_score: 0.2443 - loss: 1.4933 - precision: 0.5663 - recall: 0.1560 - val_accuracy: 0.4477 - val_f1_score: 0.3179 - val_loss: 1.4414 - val_precision: 0.5725 - val_recall: 0.2200
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 38s 162ms/step - accuracy: 0.4687 - f1_score: 0.3422 - loss: 1.4051 - precision: 0.5953 - recall: 0.2402 - val_accuracy: 0.4816 - val_f1_score: 0.3711 - val_loss: 1.3620 - val_precision: 0.6192 - val_recall: 0.2650
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 23s 99ms/step - accuracy: 0.5060 - f1_score: 0.4066 - loss: 1.3015 - precision: 0.6457 - recall: 0.2968 - val_accuracy: 0.5607 - val_f1_score: 0.4809 - val_loss: 1.1875 - val_precision: 0.6738 - val_reca

In [14]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure to avoid memory issues

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

# Save Precision plot
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

# Save Recall plot
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

# Save F1 Score plot
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot.png')  # Save the plot as a PNG file
plt.close()  # Close the figure

In [15]:
cnn_lstm.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 331,988 (1.27 MB)

 Trainable params: 110,662 (432.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 221,326 (864.56 KB)